# UMA Projekt
### Miłosz Cieśla, Filip Ryniewicz

In [4]:
import sys
sys.path.append('../')

## Przygotowanie danych

In [5]:
from ucimlrepo import fetch_ucirepo
import numpy as np

breast_cancer_wisconsin_diagnostic = fetch_ucirepo(id=17)

X = breast_cancer_wisconsin_diagnostic.data.features
X = X.values.tolist()
Y = np.array(breast_cancer_wisconsin_diagnostic.data.targets.Diagnosis)
Y = Y == "M"

### Podział na Zbiór treningowy i testowy

In [6]:
from sklearn.model_selection import train_test_split
import torch
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.5, random_state=42)
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32).view(-1, 1)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32).view(-1, 1)

## Trenowanie modeli

### Perceptron wielowarstwowy

In [7]:
from models.mlp import MLP
from time import perf_counter

mlp_model = MLP()
mlp_start_time = perf_counter()
mlp_model.train(X_train_tensor, y_train_tensor)
mlp_train_time = perf_counter() - mlp_start_time

### K najbliższych sąsiadów

In [8]:
from models.knn import KNN
from time import perf_counter

knn_model = KNN(n_neighbors=10)
knn_start_time = perf_counter()
knn_model.train(X_train, y_train)
knn_train_time = perf_counter() - knn_start_time


### Las losowy

In [9]:
from models.random_forest import RandomForest
from time import perf_counter

rf_model = RandomForest(n_estimators=100)
rf_start_time = perf_counter()
rf_model.train(X_train, y_train)
rf_train_time = perf_counter() - rf_start_time


### Czasy trenowania

In [12]:
import pandas as pd
data = {"MLP": [mlp_train_time], "KNN": [knn_train_time], "Random Forest": [rf_train_time]}
df = pd.DataFrame(data, index=["Training Time"])
df

,MLP,KNN,Random Forest
Training Time,0.028007,0.000514,0.050994


## Predykcje modeli

In [10]:
predicted = model.predict(X_test)
correct = 0
for y_pred, y_real in zip(predicted, y_test):
    if y_pred == y_real:
        correct += 1
    print(f"KNN Predicted: {y_pred}, Real: {y_real}")

NameError: name 'model' is not defined